<a href="https://colab.research.google.com/github/catarinacecilio/learning_imagery/blob/master/learning_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from PIL import Image
import sys
import csv
import multiprocessing

## Anotações


1.   Nossa tabela tem '_NaN_' e 'None'
2.   'None' é sem marcação por voluntário
3.   '_NaN_' é marcação de nenhum 
4.   Temos que compreender cada dimensão das imagens (são diferentes por pasta)
5.   Assim como alterar as dimensões das imagens para unificá-las, precisamos também fazer isso com os pontos proporcionalmente (mesmo resize)
6.   Ideia: transformar os pontos de cada imagem em uma imagem colocar os pontos de onde tem penguim como um pixel "pintado" - 1 para penguim e 0 para sem penguim
7.   Documento peng_size tem informação do tamanho do penguim em relação à câmera
8. OpenCV https://pypi.org/project/opencv-python/
9. Geometria pra ser o penguim
10. Os modelos procuram por geometrias, talvez os pontos não sejam suficientes
11. Vamos testar com os pontos e se não der certo, testar com as geometrias

# Raw Data

In [ ]:
!tar -xvzf /content/NEKOa.tgz -C /content/raw_data
!tar -xvzf /content/NEKOb.tgz -C /content/raw_data
!tar -xvzf /content/NEKOc.tgz -C /content/raw_data
!tar -xvzf /content/PETEc.tgz -C /content/raw_data
!tar -xvzf /content/CompleteAnnotations_2016-07-11.tgz -C /content/raw_data

In [ ]:
roi_df = pd.read_csv('raw_data/roi.tsv', sep='\,')
camera_names = roi_df['Camera name']
camera_names

In [ ]:
# default format can be changed as needed
def createFileList(myDir, format='.JPG'): 
    fileList = []
    print(myDir)
    labels = []
    names = []
    keywords = {"K" : "1","U": "0",} # keys and values to be changed as needed
    for root, dirs, files in os.walk(myDir, topdown=True):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
            for keyword in keywords:
                if keyword in name:
                    labels.append(keywords[keyword])
                else:
                    continue
            names.append(name)
    return fileList, labels, names

In [ ]:
# load the original image
myDir = '/content/raw_data/' #mudar
myFileList, labels, names  = createFileList(myDir)
n_files = len(names)

In [ ]:
i = 0
for file in myFileList:
    print(file)
    img_file = Image.open(file)
    # img_file.show()
# get original image parameters...
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode
# Make image Greyscale
    img_grey = img_file.convert('L')
    #img_grey.save('result.png')
    #img_grey.show()
# Save Greyscale values
    value = np.asarray(img_grey.getdata(), dtype=np.int).reshape((width, height))
    value = value.flatten()
    
    value = np.append(value,labels[i])
    i +=1
    
    print(value)
    with open("penguin_dataset.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(value)

# Função para coletar pontos nas fotos e transformar em dataframe

In [ ]:
def transform_to_list(data):
    '''Função que transforma o .json em lista de records'''
    list_of_records = []
    temp_dict = {}
  
    if data['xy'] != None :
        tamanho = 0
        index_vol = 0
        for i, voluntario in enumerate(data['xy']) :
            if voluntario != None and voluntario != '_NaN_' :
                if len(voluntario) > tamanho :
                    tamanho = len(voluntario)
                    index_vol = i
                temp_dict['imName'] = data['imName']
                temp_dict['biggest_record']  = data['xy'][i]
        list_of_records.append(temp_dict)
    
    return list_of_records

In [ ]:
def transform_to_df(data):
    '''Função que transforma o .json para dataframe'''
    
    list_of_records = transform_to_list(data)
    base_df = pd.DataFrame([], columns=['imName', 'x', 'y'])
    
    for record in list_of_records :
        if record != {} :
            if record['biggest_record'] != None :
                for xy in record['biggest_record'] :
                    temp = {}
                    temp['imName'] = record['imName']
                    if type(xy) == list and len(xy) > 1 :
                        temp['x'] = xy[0]
                        temp['y'] = xy[1]
                    elif xy != None and xy == '_NaN_' :
                        temp['x'] = 'NaN'
                        temp['y'] = 'NaN'
                    base_df = base_df.append(temp, ignore_index=True)

    return base_df

In [ ]:
# Opening JSON file
with open('sample_data/BAILa.json') as json_file:
    data_HALFb = json.load(json_file)

data_HALFb = data_HALFb['dots']

In [ ]:
#Código para processar em paralelo

with multiprocessing.Pool(4) as p:
    df_HALFb = p.map(transform_to_df, data_HALFb)


In [ ]:
#Como processamos em paralelo, o output da célula de cima é uma lista de dataframes,
# aqui vai o código de concatenar dataframes em uma só

new_df = pd.DataFrame([])

for df in df_HALFb :
    new_df = pd.concat([new_df, df], axis=0)